<h1>Import important libreary.</h1>

In [1]:
import os
import sys
import cv2
import numpy as np
from sklearn import svm
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score 
from skimage.feature import hog
import math
import tkinter as tk
from tkinter import simpledialog
import tkinter.messagebox as tkMessageBox

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


<h1>This function is  for getting person name. </h1>

In [47]:
application_window = tk.Tk()
application_window.withdraw()
def getClassName():
    return simpledialog.askstring("Input", "Please enter the Person name.\nIf you write exit then the program will close. \nPress Ok or Enter to close the program.", parent=application_window)

<h1>In this part, we will take training image from webcam.</h1>

In [49]:
folder = 'img'
cascPath = 'haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)
# Read the image
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_EXPOSURE, 1) 

while(1):
    ret, image = cap.read()


    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.FONT_HERSHEY_SIMPLEX
    )

   # print ("Found {0} faces!".format(len(faces)))

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        t = time.localtime()
        roi=image[y:y+h, x:x+w]
        cv2.imwrite("image/"+folder+'/'+str(time.asctime(t))+".png",roi)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.imshow("Webcam", image)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        folder = getClassName()
        if str(folder) == "exit" or len(str(folder)) == 0:
            break
        if(len(folder) > 0 ):
            if not os.path.exists('image/'+folder):
                os.makedirs('image/'+folder)
cv2.destroyAllWindows()
cap.release()    




<h1>This part is for image preprocessing.</h1>

In [2]:
all_dir = []
for _, dirnames, filenames in os.walk('image'):
    all_dir.append(dirnames)
image = []
classs = []
for i in (all_dir[0]):
     for _, dirnames, filenames in os.walk('image/'+i):
        for j in filenames:
            img = cv2.imread('image/'+i+'/'+j)
            img = cv2.resize(img, (180, 180))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            equ = cv2.equalizeHist(gray)
            
            fd,hog_image = hog(equ, orientations=8, pixels_per_cell=(16,16),cells_per_block=(4, 4),block_norm= 'L2',visualise=True)
            
            
            
            a = np.array(fd)
            a = a.flatten()
            image.append(a)
            classs.append(i)
np.array(image).shape

/usr/local/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


(112, 8192)

<h1>Train model</h1>

In [3]:
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(image, classs)
print(clf.best_estimator_)



SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


<h1>Test Model</h1>

In [6]:
font = cv2.FONT_HERSHEY_SIMPLEX
cascPath = 'haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_EXPOSURE, 1) 
while(1):
    ret, image = cap.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.FONT_HERSHEY_SIMPLEX
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        
        roi=gray[y:y+h, x:x+w]
        img = cv2.resize(roi, (180, 180))
        equ = cv2.equalizeHist(img)
        fd,hog_image = hog(equ, orientations=8, pixels_per_cell=(16,16),cells_per_block=(4, 4),block_norm= 'L2',visualise=True)
        #ridges = ridge_filter.getRidgeFilteredImage(equ)
        feature = np.array(fd)
        feature = feature.flatten()
        y_pred = clf.predict([feature])
        prob = (clf.predict_proba([feature]))
        max_prob = max(prob[0])
        
        if(max_prob > .2000):
            cv2.putText(image, y_pred[0]+str((max_prob*100))[:2]+"%", (x,y-20), font, 1, (0,255,0), 1, cv2.LINE_AA)
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.imshow("Webcam", image)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        cv2.destroyAllWindows()
        cap.release()    
        break
cv2.destroyAllWindows()
cap.release()    


/usr/local/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)
